In [ ]:
import pandas as pd
import pickle 
import numpy as np
import os
import sys
sys.path.append('/home/stankeaa/wikiwho_inheritance/')
from sequencers.clusterers import kmeans, dbscan, token_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from sequencers import tsne

In [ ]:
dirs = []
for file in os.listdir( sys.path[-1] ):
    if re.match("^[0-9_-]*$", file):
        dirs.append(file)

In [ ]:
vecs = []
for item in os.listdir(sys.path[-1] + a.value + '/vectorizers/'):
    if item != '.ipynb_checkpoints':
        vecs.append(item.split('___')[0])

In [ ]:
vecs

## Clusterer
This notebook clusterizes wikiwho chobs. 
It requires an **existing pickle** with chobs already vectorized (e.g. with **features** added). <br>
The following clustering methods are possible: 
- Kmeans (parameter: *random state*)
- DBscan (parameters: *min number of samples in one cluster* and *eps*)
- Token similarity (parameter: *intersection*, or min number of tokens to be the same in context)

In [ ]:
a = widgets.Dropdown(
    options = dirs,
    value = None,
    description = 'Article id:',
)
v = widgets.Dropdown(
    options = [],
    value = None,
    description = 'Vectorizer:',
)
c = widgets.Dropdown(
    options = ['kmeans.Kmeans', 'dbscan.DBscan', 'token_similarity.Token_Similarity'],
    description = 'Clusterer:',
)
p1 = widgets.IntText(
    value=None
)
p2 = widgets.FloatText(
    value=None
)
p3 = widgets.Dropdown(
    options = ['id', 'string'],
    description = 'Token type:',
)
ts = widgets.IntText(
    value=42,
    description = 'TSNE random state:',
)

def a_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        v.options = os.listdir(sys.path[-1] + change['new'] + '/vectorizers/')
        display(v, c, ts)
        
def v_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global vectorizer
        vectorizer = change['new']

def c_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global clusterer
        clusterer = eval(c.value)
        clear_output()
        display(a, v, c, ts)
        if change['new'] == 'kmeans.Kmeans':
            p1.value = 42
            p1.description = 'Random state: '
            display(p1)
        elif change['new'] == 'token_similarity.Token_Similarity':
            p1.value = 2
            p1.description = 'Intersection: '
            display(p1, p3)
        else:
            p1.value = 4
            p1.description = 'Min samples: '
            p2.value = 1.5
            p2.description = 'Eps: '
            display(p1, p2)
        print('Data is saved')

a.observe(a_on_change)
v.observe(v_on_change)
c.observe(c_on_change)
display(a)

In [ ]:
save_path = '../'+a.value +'/clusterers/'+vectorizer[:-4]
if not os.path.exists(save_path):
    os.mkdir(save_path+'/') 
if clusterer == kmeans.Kmeans:
    cl = clusterer(pd.read_pickle('../'+a.value +'/vectorizers/'+vectorizer), {'random_state':p1.value}, save_path)
elif clusterer == token_similarity.Token_Similarity:
    cl = clusterer(pd.read_pickle('../'+a.value +'/vectorizers/'+vectorizer), {'intersection':p1.value, 'token_type':p3.value}, save_path)
else:
    cl = clusterer(pd.read_pickle('../'+a.value +'/vectorizers/'+vectorizer), {'min_samples':p1.value, 'eps':p2.value}, save_path)
cl.get_clusters()
cl.save()

In [ ]:
if not os.path.exists(save_path+'/tsne/'):
    os.mkdir(save_path+'/tsne/') 
ft = tsne.Tsne(pd.read_pickle(cl.dirpath + '.pkl'), {'random_state':ts.value}, save_path+'/tsne/'+cl.dirpath.split('/')[-1])
ft.dirpath = ft.path
ft.get_plot_data(cl.df)
ft.save()

In [ ]:
strin_clean = pd.read_pickle('../39570/clusterers/Word_Embed_word_embed_size_300_use_gap_True_context_5___Chobs_context_5_gap_length_20/tsne/Token_Similarity_intersection_2_token_type_string.pkl')

In [ ]:
strin_dirty = pd.read_pickle('../39570/clusterers/Word_Embed_word_embed_size_300_use_gap_True_left_context_5_right_context_5/tsne/Token_Similarity_intersection_2_token_type_string.pkl')

In [ ]:
ids['clusters']
#ids.columns

In [ ]:
strin['clusters']

In [ ]:
strin_dirty['clusters']